# Prompt Engineering for HybridRAG

Master system prompts and reranking strategies.

**What you'll learn:**
- Query type detection
- Reranking instructions
- Custom prompt creation
- Citation handling
- Response quality optimization

**Prerequisites:** Completed notebooks 01-02

In [ ]:
# Setup
from dotenv import load_dotenv

from hybridrag import SYSTEM_PROMPT, SYSTEM_PROMPT_COMPACT, create_hybridrag
from hybridrag.prompts import (
    QueryType,
    create_system_prompt,
    detect_query_type,
    select_rerank_instruction,
)

load_dotenv()
rag = await create_hybridrag()
print("✓ HybridRAG initialized")

## 1. System Prompts Overview

In [ ]:
# Default system prompt (full version)
print("FULL System Prompt:")
print("=" * 60)
print(SYSTEM_PROMPT)
print("\n" + "=" * 60)
print(f"Length: {len(SYSTEM_PROMPT)} characters\n")

# Compact system prompt
print("COMPACT System Prompt:")
print("=" * 60)
print(SYSTEM_PROMPT_COMPACT)
print("\n" + "=" * 60)
print(f"Length: {len(SYSTEM_PROMPT_COMPACT)} characters\n")

print(
    f"Token savings: {len(SYSTEM_PROMPT) - len(SYSTEM_PROMPT_COMPACT)} characters (~{(len(SYSTEM_PROMPT) - len(SYSTEM_PROMPT_COMPACT)) // 4} tokens)"
)

## 2. Query Type Detection

In [ ]:
# Test different query types
test_queries = [
    "What is MongoDB Atlas?",
    "Summarize the benefits of vector search",
    "Show me how to create a vector index",
    "Why is my query slow?",
    "mongodb atlas vector search setup",
]

print("Query Type Detection:\n")

for query in test_queries:
    query_type = detect_query_type(query)
    print(f"Query: {query}")
    print(f"Type: {query_type.value}")
    print(f"Pattern matched: {get_pattern_hint(query)}")
    print()


def get_pattern_hint(query: str) -> str:
    """Show which pattern triggered the detection"""
    query_lower = query.lower()
    if any(
        word in query_lower for word in ["summary", "summarize", "explain", "overview"]
    ):
        return "Summary keywords"
    elif any(
        word in query_lower for word in ["how", "guide", "tutorial", "example", "show"]
    ):
        return "How-to keywords"
    elif any(
        word in query_lower
        for word in ["error", "issue", "problem", "why", "troubleshoot"]
    ):
        return "Troubleshooting keywords"
    else:
        return "General query"

## 3. Reranking Instructions

In [ ]:
# Get reranking instructions for different query types
for query_type in QueryType:
    instruction = select_rerank_instruction(query_type)
    print(f"Query Type: {query_type.value}")
    print(f"Reranking Instruction:\n{instruction}")
    print("\n" + "=" * 60 + "\n")

## 4. Custom Domain Prompts

In [ ]:
# Create custom prompts for different domains

# Technical documentation domain
tech_docs_prompt = create_system_prompt(
    domain="MongoDB Atlas technical documentation",
    response_guidelines=[
        "Provide code examples when relevant",
        "Link to official documentation",
        "Mention version compatibility",
    ],
    citation_style="Include document URLs and version numbers",
)

print("Technical Docs Prompt:")
print(tech_docs_prompt)
print("\n" + "=" * 60 + "\n")

# Customer support domain
support_prompt = create_system_prompt(
    domain="customer support for MongoDB services",
    response_guidelines=[
        "Be empathetic and patient",
        "Provide step-by-step troubleshooting",
        "Escalate to human if necessary",
    ],
    citation_style="Reference knowledge base article IDs",
)

print("Customer Support Prompt:")
print(support_prompt)

## 5. Citation Handling

In [ ]:
# Query with citation requirements
query = "How does vector search work in MongoDB?"

# Get answer with citations
answer = await rag.query_with_answer(
    query=query,
    mode="hybrid",
    top_k=3,
)

print(f"Query: {query}\n")
print("Answer with citations:")
print(answer)
print("\n" + "=" * 60 + "\n")

# Note: The SYSTEM_PROMPT instructs the LLM to:
# 1. Use [1], [2], [3] format for citations
# 2. Include all sources used
# 3. Format as: "[X] Source: <metadata>"

## 6. Response Quality Comparison

In [ ]:
query = "Explain hybrid search"

# Test with different prompts
prompts = {
    "Default": SYSTEM_PROMPT,
    "Compact": SYSTEM_PROMPT_COMPACT,
    "Technical": tech_docs_prompt,
}

print(f"Query: {query}\n")

for name, prompt in prompts.items():
    print(f"\n{'=' * 60}")
    print(f"Prompt Type: {name}")
    print(f"{'=' * 60}\n")

    # Note: In practice, you'd pass the custom prompt to the LLM
    # For this demo, we'll show the prompt characteristics
    print(f"Prompt length: {len(prompt)} characters")
    print(f"First 200 chars: {prompt[:200]}...")
    print("\nKey instructions:")
    if "code examples" in prompt.lower():
        print("  ✓ Includes code examples")
    if "citation" in prompt.lower():
        print("  ✓ Citation formatting")
    if "step-by-step" in prompt.lower():
        print("  ✓ Step-by-step guidance")
    print()

## 7. Adaptive Prompting

In [ ]:
def adaptive_prompt(query: str) -> str:
    """Select prompt based on query characteristics"""
    query_type = detect_query_type(query)

    # Use compact for simple queries
    if query_type == QueryType.GENERAL and len(query.split()) < 5:
        return SYSTEM_PROMPT_COMPACT

    # Use technical for how-to queries
    elif query_type == QueryType.TOOLS:
        return create_system_prompt(
            domain="technical documentation",
            response_guidelines=["Include code examples", "Be precise and technical"],
        )

    # Use empathetic for troubleshooting
    elif query_type == QueryType.TROUBLESHOOTING:
        return create_system_prompt(
            domain="customer support",
            response_guidelines=[
                "Be helpful and patient",
                "Provide step-by-step solutions",
            ],
        )

    # Default
    else:
        return SYSTEM_PROMPT


# Test adaptive prompting
test_queries = [
    "What is MongoDB?",
    "How do I create a vector index?",
    "My query is returning no results, why?",
    "Summarize the benefits of Atlas Search",
]

print("Adaptive Prompt Selection:\n")

for query in test_queries:
    selected_prompt = adaptive_prompt(query)
    query_type = detect_query_type(query)

    print(f"Query: {query}")
    print(f"Type: {query_type.value}")
    print(
        f"Selected prompt: {'COMPACT' if selected_prompt == SYSTEM_PROMPT_COMPACT else 'CUSTOM' if selected_prompt != SYSTEM_PROMPT else 'DEFAULT'}"
    )
    print(f"Prompt length: {len(selected_prompt)} chars\n")

## 8. Best Practices

### Prompt Selection:

- **Default (SYSTEM_PROMPT)**: General queries, when unsure
- **Compact (SYSTEM_PROMPT_COMPACT)**: Simple queries, token budget constraints
- **Custom**: Domain-specific requirements

### Query Type Optimization:

- **GENERAL**: Standard prompt, balanced approach
- **SUMMARY**: Emphasize conciseness and key points
- **TOOLS**: Include code examples and technical details
- **TROUBLESHOOTING**: Be empathetic, provide step-by-step guidance

### Citation Quality:

- Always include source attribution
- Use consistent citation format ([1], [2], etc.)
- Provide enough metadata for verification

### Response Guidelines:

- Keep instructions clear and actionable
- Avoid contradictory guidelines
- Test prompts with real queries
- Iterate based on user feedback

## Next Steps

- `05_performance_tuning.ipynb` - Production optimization
- **Further reading:**
  - Prompt Engineering Best Practices
  - LLM Response Quality Evaluation
  - Citation and Attribution Standards